## Importing the required libraries

In [1]:
import numpy as np
import matplotlib as plt
import pandas as pd
import random
import sklearn
import math
from sklearn.linear_model import LinearRegression
from scipy.stats import multivariate_normal

#Getting the data ready

In [2]:
#Creating a dataframe using the data provided in the table and adding noise to 
# the Salary column, with the noise being Normal distribution

header=["Gender_ID","Degree","Age","Salary"]

#The noise array with five elements, is be added to the "Salary" column.
noise=np.random.normal(0,1,5) 

#Data as given in table 8.2 of the book
training_data=[
    [-1,-1,+1,-1,+1],
    [2,3,1,1,2],
    [36,47,26,68,33],
    [89.563,123.543,23.989,138.769,113.888]
]

df=pd.DataFrame(training_data).transpose()
df.columns=header

#Adding noise to the data.
df["Salary"]=[df["Salary"][i]+noise[i] for i in range(len(df))]
df.head()
#Switching the data type of each column to integer, except the salary colum
data_types={"Gender_ID":int,"Degree":int,"Age":int}
df.astype(data_types)

,Gender_ID,Degree,Age,Salary
0,-1,2,36,88.998379
1,-1,3,47,125.585021
2,1,1,26,23.918151
3,-1,1,68,138.998072
4,1,2,33,112.882847


In [3]:
#Making three different attribute datasets: 

#First with only the "Age" column
X1=np.array(df[["Age"]]).reshape(-1,1)
X1=X1.astype("int")

#Second with the "Age" and "Degree" columns
X2=np.array(df[["Age","Degree"]]).reshape(-1,2)
X2=X2.astype("int")

#Third with the "Age","Degree" and the "Gender_ID" columns
X3=np.array(df[["Age","Degree","Gender_ID"]]).reshape(-1,3)
X3=X3.astype("int")

#the column with the "Salary" values.
Y=np.array(df["Salary"]).reshape(-1,1)

#Straight Curve Fitting Regression Model
For straight curve fitting regression, the existing LinearRegression library has been used

In [4]:
#The regression model in this cell is for data with only "Age" column.
regr=LinearRegression()
regr.fit(X1,Y)

#The score evaluating the linear fit for X1 data.
score_SCF_X1=regr.score(X1,Y)

#prediction_SCF_X1 is the prediction by the model for X1 
#using the straight curve fitting method. 
#It has the five predictions made for each of the
#five rows from the training data set
prediction_SCF_X1=regr.predict(np.array(X1))
prediction_SCF_X1=[prediction_SCF_X1[i][0] for i in range(len(X1))]

print("The prediction by the SCF model on data in X1:")
print(prediction_SCF_X1)
print("\n")

#This gives the prediction of salary for age 60, as asked in the question
y_test=regr.predict(np.array([[60]]))
print("The salary predicted by the straight curve when the age is 60 is:",y_test[0][0])

The prediction by the SCF model on data in X1:
[85.17566353936193, 108.82718633792182, 63.67427917703478, 153.98009349880886, 78.72524823066378]


The salary predicted by the straight curve when the age is 60 is: 136.77898600894713


The following two cells are the regression fits for the data stored in X2, X3. 

In [5]:
#The regression model in this cell is for data with "Age" and "Degree" columns.
regr=LinearRegression()
regr.fit(X2,Y)

#The score evaluating the linear fit for X2 data.
score_SCF_X2=regr.score(X2,Y)

#prediction_SCF_X2 is the prediction by the model for X2
#using the straight curve fitting method. 
#It has the five predictions made for each of the
#five rows from the training data set
prediction_SCF_X2=regr.predict(np.array(X2))
prediction_SCF_X2=[prediction_SCF_X2[i][0] for i in range(len(X2))]

print("The prediction by the SCF model on data in X2:")
print(prediction_SCF_X2)

The prediction by the SCF model on data in X2:
[89.60610345275774, 140.3245454191955, 41.15767113233068, 136.49807626478176, 82.7960745147255]


In [6]:
#The regression model in this cell is for data with "Age", "Degree" and 
# "Gender_ID" columns.
regr=LinearRegression()
regr.fit(X3,Y)

#The score evaluating the linear fit for X3 data.
score_SCF_X3=regr.score(X3,Y)

#prediction_SCF_X2 is the prediction by the model for X2
#using the straight curve fitting method. 
#It has the five predictions made for each of the
#five rows from the training data set
prediction_SCF_X3=regr.predict(np.array(X3))
prediction_SCF_X3=[prediction_SCF_X3[i][0] for i in range(len(X3))]

print("The prediction by the SCF model on data in X3:")
print(prediction_SCF_X3)

The prediction by the SCF model on data in X3:
[74.91926776794779, 141.2731738039828, 41.21534455614375, 137.38903122575596, 95.58565342996081]


# Maximum Likelihood Estimation

In [7]:
#function to calculate the likelihood function, given the predicted outcomes, 
#the actual values and the number of samples. 
#It returns the log of the probability.

def calcLogLikelihood(prediction,true,n):
    #error column contains the difference between
    #the predicted values and the true values.
    error=true-prediction 
    sigma=np.std(error)
    #the formula for the likelihood
    formula=((1.0/(2.0*math.pi*(sigma**2)))**(n/2))* \
        np.exp(-1*((np.dot(error.T,error))/(2*sigma*sigma)))
    return np.log(formula)

In [8]:
#This cell has functions for three different models, corresponding to 
#each of the datasets. 
#They take in the variable array and return the negative loglikelihood 
#value for that set of variables.

def MLE_model1(var):
    x,y=np.array(df["Age"]),np.array(df["Salary"])
    #predicts the value of y on the basis of the X column
    y_prediction=np.array([(var[1]*(x[i])+var[0]) for i in range(len(x))])
    likely=calcLogLikelihood(y_prediction,y,float(len(y_prediction)))
    return (-1*likely)

def MLE_model2(var):
    x1,x2,y=df["Age"],df["Degree"],df["Salary"]
    #initializing the array
    y_prediction=[0 for i in range(len(x1))]
    #predicts the value of y on the basis of the X columns
    y_prediction=[(var[2]*x2[i]+var[1]*x1[i]+var[0]) for i in range(len(x1))]
    likely=calcLogLikelihood(y_prediction,y,float(len(y_prediction)))
    return (-1*likely)

def MLE_model3(var):
    x1,x2,x3,y=df["Age"],df["Degree"],df["Gender_ID"],df["Salary"]
    #initializing the array
    y_prediction=[0 for i in range(len(x1))]
    #predicts the value of y on the basis of the X columns
    y_prediction=[(var[3]*x3[i]+var[2]*x2[i]+var[1]*x1[i]+var[0]) for i in range(len(x1))]
    likely=calcLogLikelihood(y_prediction,y,float(len(y_prediction)))
    return (-1*likely)

The next three cells are for the estimation of the parameters using the likelihood function.  
We do so minimizing the -log  likelihood function instead of maximizing the loglikelihood function. In this way the MLE method is implemented.

In [9]:
# For data with "Age" column.
from scipy.optimize import minimize
nvar=2
var=np.zeros(nvar)
#Randomly picked 2 variable inputs.
var[0]=2
var[1]=3

#This is the minimizing function.
res1=minimize(MLE_model1,var,method="BFGS",options={"disp":False})
#print(res1.x) #res.x gives the parameters in an array

#Making the prediction
theta_MLE_1=res1.x
prediction_MLE_X1=[theta_MLE_1[0]+theta_MLE_1[1]*X1[i][0] for i in range(len(X1))]
print("The prediction by the MLE model on data X1:")
print(prediction_MLE_X1)
#print(prediction_SCF_X1)

The prediction by the MLE model on data X1:
[85.17568666180486, 108.82718264332831, 63.674326678601716, 153.9800386080549, 78.7252786668439]


In [10]:
# For data with "Age", "Degree" columns.
from scipy.optimize import minimize
nvar=3
var=np.zeros(nvar)
#Randomly picked 3 variable inputs.
var[0]=2
var[1]=3
var[2]=5

#This is the minimizing function.
res2=minimize(MLE_model2,var,method="BFGS",options={"disp":False})
#print(res2.x)

#Making the prediction
theta_MLE_2=res2.x
prediction_MLE_X2=[theta_MLE_2[0]+theta_MLE_2[1]*X2[i][0] + theta_MLE_2[2]*X2[i][1] for i in range(len(X2))]
print("The prediction by the MLE model on data X2:")
print(prediction_MLE_X2)
#print(prediction_SCF_X2)

The prediction by the MLE model on data X2:
[89.60611152581689, 140.32454602616554, 41.15768595103656, 136.498060824907, 82.79608474911186]


In [11]:
# For data with "Age", "Degree" and "Gender_ID" columns.
from scipy.optimize import minimize
nvar=4
var=np.zeros(nvar)
#Randomly picked 3 variable inputs.
var[0]=2
var[1]=3
var[2]=5
var[3]=7

#This is the minimizing function.
res3=minimize(MLE_model3,var,method="BFGS",options={"disp":False})
#print(res3.x)

#Making the prediction
theta_MLE_3=res3.x
prediction_MLE_X3=[theta_MLE_3[0]+theta_MLE_3[1]*X3[i][0] + theta_MLE_3[2]*X3[i][1] + theta_MLE_3[3]*X3[i][2] for i in range(len(X3))]
print("The prediction by the MLE model on data X3:")
print(prediction_MLE_X3)
#print(prediction_SCF_X3)

The prediction by the MLE model on data X3:
[74.91927288115535, 141.27317934330674, 41.21535205048788, 137.3890280598322, 95.58566208093428]


#MAP Estimate 

In [12]:
#This cell has the 3 MAP models with the probability distribution due to the 
#parameters added to the log likelihood calculated in the previous section.
#The distribution has mean zero and variance is given by the
#Covariance matrix (Identity matrix multiplied by a constant)

def MAP_model1(var):
    n=len(var)
    x,y=np.array(df["Age"]),np.array(df["Salary"])
    #predicts the value of y on the basis of the X column
    y_prediction=np.array([(var[1]*(x[i])+var[0]) for i in range(len(x))])
    
    #The covariance matrix
    cov_mat=np.identity(n,dtype=int)*2
    var_dist=multivariate_normal.pdf(var,[0]*n,cov_mat)
    
    #Adding the log of the probability for that set of parameters
    likely=calcLogLikelihood(y_prediction,y,float(len(y_prediction)))+np.log(var_dist)
    return (-1*likely)

def MAP_model2(var):
    n=len(var)
    x1,x2,y=df["Age"],df["Degree"],df["Salary"]
    #initializing the array
    y_prediction=[0 for i in range(len(x1))]
    #predicts the value of y on the basis of the X columns
    y_prediction=[(var[2]*x2[i]+var[1]*x1[i]+var[0]) for i in range(len(x1))]
    
    #The covariance matrix
    cov_mat=np.identity(n,dtype=int)*2
    var_dist=multivariate_normal.pdf(var,[0]*n,cov_mat)
    
    #Adding the log of the probability for that set of parameters
    likely=calcLogLikelihood(y_prediction,y,float(len(y_prediction)))+np.log(var_dist)
    return (-1*likely)

def MAP_model3(var):
    n=len(var)
    x1,x2,x3,y=df["Age"],df["Degree"],df["Gender_ID"],df["Salary"]
    y_prediction=[0 for i in range(len(x1))]
    #predicts the value of y on the basis of the X columns
    y_prediction=[(var[3]*x3[i]+var[2]*x2[i]+var[1]*x1[i]+var[0]) for i in range(len(x1))]
    
    #The covariance matrix
    cov_mat=np.identity(n,dtype=int)*2
    var_dist=multivariate_normal.pdf(var,[0]*n,cov_mat)
    
    #Adding the log of the probability for that set of parameters
    likely=calcLogLikelihood(y_prediction,y,float(len(y_prediction)))+np.log(var_dist)
    return (-1*likely)

The following three cells are the models for minimizing the loss and predicting the results.

In [13]:
# For data with "Age" column.
from scipy.optimize import minimize
nvar=2
var=np.zeros(nvar)
#Randomly picked 2 variable inputs.
var[0]=2
var[1]=3

#This is the minimizing function.
res1=minimize(MAP_model1,var,method="BFGS",options={"disp":False})
#print(res1.x)

#Making the prediction
theta_MAP_1=res1.x
prediction_MAP_X1=[theta_MAP_1[0]+theta_MAP_1[1]*X1[i][0] for i in range(len(X1))]
print("The prediction by the MAP model on data X1:")
print(prediction_MAP_X1)
#print(prediction_SCF_X1)
#print(Y)

The prediction by the MAP model on data X1:
[80.69529094756592, 105.33380080214222, 58.29664562522384, 152.3709559790606, 73.9756973508633]


In [14]:
# For data with "Age", "Degree"columns.
from scipy.optimize import minimize
nvar=3
var=np.zeros(nvar)
#Randomly picked 3 variable inputs.
var[0]=2
var[1]=3
var[2]=5

#This is the minimizing function.
res2=minimize(MAP_model2,var,method="BFGS",options={"disp":False})
#print(res2.x)

#Making the prediction
theta_MAP_2=res2.x
#print(res2.x)
prediction_MAP_X2=[theta_MAP_2[0]+theta_MAP_2[1]*X2[i][0] + theta_MAP_2[2]*X2[i][1] for i in range(len(X2))]
print("The prediction by the MAP model on data X2:")
print(prediction_MAP_X2)
#print(prediction_SCF_X2)
#print(Y)

The prediction by the MAP model on data X2:
[80.95164989338468, 105.7937788382329, 58.33868618018479, 151.96362590941328, 74.2641541984398]


In [15]:
# For data with "Age", "Degree" and "Gender_ID" columns.
from scipy.optimize import minimize
nvar=4
var=np.zeros(nvar)
#Randomly picked 4 variable inputs.
var[0]=2
var[1]=3
var[2]=5
var[3]=7

#This is the minimizing function.
res3=minimize(MAP_model3,var,method="BFGS",options={"disp":False})
#print(res3.x)

#Making the prediction
theta_MAP_3=res3.x
prediction_MAP_X3=[theta_MAP_3[0]+theta_MAP_3[1]*X3[i][0] + theta_MAP_3[2]*X3[i][1] + theta_MAP_3[3]*X3[i][2] for i in range(len(X3))]
print("The prediction by the MAP model on data X3:")
print(prediction_MAP_X3)
#print(prediction_SCF_X3)
#print(Y)

The prediction by the MAP model on data X3:
[80.97397214831902, 105.8130818595535, 58.29938056417188, 151.976947922029, 74.22294743959758]


# Comparison and Summary

In [16]:
y=[Y[i][0] for i in range(len(Y))]
print("The correct values of y, as given in the data are:",y)

The correct values of y, as given in the data are: [88.99837884802459, 125.58502145761152, 23.918150943477933, 138.9980724920505, 112.8828470426266]


In [17]:
print("When 'Age' is the only parameter:")
print("\t Predictions by the Straight curve fit model are:", prediction_SCF_X1)
print("\t Predictions by the MLE model are:", prediction_MLE_X1)
print("\t Predictions by the MAP model are:", prediction_MAP_X1)

When 'Age' is the only parameter:
	 Predictions by the Straight curve fit model are: [85.17566353936193, 108.82718633792182, 63.67427917703478, 153.98009349880886, 78.72524823066378]
	 Predictions by the MLE model are: [85.17568666180486, 108.82718264332831, 63.674326678601716, 153.9800386080549, 78.7252786668439]
	 Predictions by the MAP model are: [80.69529094756592, 105.33380080214222, 58.29664562522384, 152.3709559790606, 73.9756973508633]


In [18]:
print("When 'Age', 'Degree' are the parameters:")
print("\t Predictions by the Straight curve fit model are:", prediction_SCF_X2)
print("\t Predictions by the MLE model are:", prediction_MLE_X2)
print("\t Predictions by the MAP model are:", prediction_MAP_X2)

When 'Age', 'Degree' are the parameters:
	 Predictions by the Straight curve fit model are: [89.60610345275774, 140.3245454191955, 41.15767113233068, 136.49807626478176, 82.7960745147255]
	 Predictions by the MLE model are: [89.60611152581689, 140.32454602616554, 41.15768595103656, 136.498060824907, 82.79608474911186]
	 Predictions by the MAP model are: [80.95164989338468, 105.7937788382329, 58.33868618018479, 151.96362590941328, 74.2641541984398]


In [19]:
print("When 'Age', 'Degree' and 'Gender_ID' are the parameters:")
print("\t Predictions by the Straight curve fit model are:", prediction_SCF_X3)
print("\t Predictions by the MLE model are:", prediction_MLE_X3)
print("\t Predictions by the MAP model are:", prediction_MAP_X3)

When 'Age', 'Degree' and 'Gender_ID' are the parameters:
	 Predictions by the Straight curve fit model are: [74.91926776794779, 141.2731738039828, 41.21534455614375, 137.38903122575596, 95.58565342996081]
	 Predictions by the MLE model are: [74.91927288115535, 141.27317934330674, 41.21535205048788, 137.3890280598322, 95.58566208093428]
	 Predictions by the MAP model are: [80.97397214831902, 105.8130818595535, 58.29938056417188, 151.976947922029, 74.22294743959758]


The predictions for the MLE and the straight curve fitting model are the same. MAP models give slightly different answers. The predictions for the MAP models also depend on the value of the lambda for the covariance matrix.

In [20]:
sq_error_SCF_X1=[(prediction_SCF_X1[i]-y[i])**2 for i in range(len(y))]
mean_sq_error_SCF_X1=(sum(sq_error_SCF_X1)/len(y))
print("The Mean squared error for SCF Model using data in X1 is: ", mean_sq_error_SCF_X1)
sq_error_MLE_X1=[(prediction_MLE_X1[i]-y[i])**2 for i in range(len(y))]
mean_sq_error_MLE_X1=(sum(sq_error_MLE_X1)/len(y))
print("The Mean squared error for MLE Model using data in X1 is: ", mean_sq_error_MLE_X1)
sq_error_MAP_X1=[(prediction_MAP_X1[i]-y[i])**2 for i in range(len(y))]
mean_sq_error_MAP_X1=(sum(sq_error_MAP_X1)/len(y))
print("The Mean squared error for MAP Model using data in X1 is: ", mean_sq_error_MAP_X1)

The Mean squared error for SCF Model using data in X1 is:  653.4380864797513
The Mean squared error for MLE Model using data in X1 is:  653.4380864811001
The Mean squared error for MAP Model using data in X1 is:  670.7068826393009


In [21]:
sq_error_SCF_X2=[(prediction_SCF_X2[i]-y[i])**2 for i in range(len(y))]
mean_sq_error_SCF_X2=(sum(sq_error_SCF_X2)/len(y))
print("The Mean squared error for SCF Model using data in X2 is: ", mean_sq_error_SCF_X2)
sq_error_MLE_X2=[(prediction_MLE_X2[i]-y[i])**2 for i in range(len(y))]
mean_sq_error_MLE_X2=(sum(sq_error_MLE_X2)/len(y))
print("The Mean squared error for MLE Model using data in X2 is: ", mean_sq_error_MLE_X2)
sq_error_MAP_X2=[(prediction_MAP_X2[i]-y[i])**2 for i in range(len(y))]
mean_sq_error_MAP_X2=(sum(sq_error_MAP_X2)/len(y))
print("The Mean squared error for MAP Model using data in X2 is: ", mean_sq_error_MAP_X2)

The Mean squared error for SCF Model using data in X2 is:  285.25756288663786
The Mean squared error for MLE Model using data in X2 is:  285.25756288676354
The Mean squared error for MAP Model using data in X2 is:  660.1450779364118


In [22]:
sq_error_SCF_X3=[(prediction_SCF_X3[i]-y[i])**2 for i in range(len(y))]
mean_sq_error_SCF_X3=(sum(sq_error_SCF_X3)/len(y))
print("The Mean squared error for SCF Model using data in X1 is: ", mean_sq_error_SCF_X3)
sq_error_MLE_X3=[(prediction_MLE_X3[i]-y[i])**2 for i in range(len(y))]
mean_sq_error_MLE_X3=(sum(sq_error_MLE_X3)/len(y))
print("The Mean squared error for MLE Model using data in X3 is: ", mean_sq_error_MLE_X3)
sq_error_MAP_X3=[(prediction_MAP_X3[i]-y[i])**2 for i in range(len(y))]
mean_sq_error_MAP_X3=(sum(sq_error_MAP_X3)/len(y))
print("The Mean squared error for MAP Model using data in X3 is: ", mean_sq_error_MAP_X3)

The Mean squared error for SCF Model using data in X1 is:  209.06286407856547
The Mean squared error for MLE Model using data in X3 is:  209.0628640786055
The Mean squared error for MAP Model using data in X3 is:  660.0857386651198


**CONCLUSION**

The MLE and Straight Curve fitting models give almost the same result. The MAP model, on the other hand, gives a different prediction. This is because the results from the MAP model also depend on the value of lambda in the covariance matrix. 
While the MAP model is expected to perform better, in this case, the MLE and SCF models gives a lower loss and therefore, a better result. 
This could be because the function does not have a zero-one loss in the case of MAP Estimation.